In [1]:
conda activate pggb_fatica


(pggb_fatica) 


: 1

In [2]:
chromosomeName=$(odgi paths -L -i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og|grep "9852")

(pggb_fatica) 


: 1

In [3]:
cat neighborhoods/final/final_all_neighborhoods.chr4.bed|grep "at9852"|sed "s/at9852_1_chr4/${chromosomeName}/g" > neighborhoods/all_neighborhoods_at9852.chr4.txt

(pggb_fatica) 


: 1

In [4]:
cat neighborhoods/all_neighborhoods_at9852.chr4.txt

at9852_1_chr4:0-24203097	1700501	1727126	chr4_e_r4_nh01
at9852_1_chr4:0-24203097	2445970	2452378	chr4_e_r3_nh01
at9852_1_chr4:0-24203097	2455797	2540938	chr4_nh01
at9852_1_chr4:0-24203097	10948767	10982024	chr4_nh02
at9852_1_chr4:0-24203097	11500339	11560875	chr4_e_r1_nh01
at9852_1_chr4:0-24203097	12195399	12210412	chr4_e_r2_nh01
at9852_1_chr4:0-24203097	12364417	12388850	chr4_nh04
at9852_1_chr4:0-24203097	12479245	12499773	chr4_e_r1_nh02
at9852_1_chr4:0-24203097	12792076	12836849	chr4_nh05
at9852_1_chr4:0-24203097	13906683	14017133	chr4_nh06
at9852_1_chr4:0-24203097	15114744	15329424	chr4_nh07
at9852_1_chr4:0-24203097	16096947	16115246	chr4_nh08
at9852_1_chr4:0-24203097	16271612	16322663	chr4_nh09
at9852_1_chr4:0-24203097	16467673	16481949	chr4_nh10
at9852_1_chr4:0-24203097	17231162	17255325	chr4_nh11
at9852_1_chr4:0-24203097	17843912	17887958	chr4_nh12
at9852_1_chr4:0-24203097	18827932	18835823	chr4_nh13
at9852_1_chr4:0-24203097	19221428	19242388	chr4_nh14
at9852_1_chr4:0-24203097	21

: 1

In [5]:
mkdir -p neighborhoods/chr4/cluster_og
mkdir -p neighborhoods/chr4/cluster_gfa
mkdir -p neighborhoods/chr4/cluster_allPathCoordinates
mkdir -p neighborhoods/chr4/cluster_viz
mkdir -p neighborhoods/chr4/cluster_pav
mkdir -p neighborhoods/chr4/cluster_similarity
mkdir -p neighborhoods/chr4/cluster_jumps
mkdir -p neighborhoods/chr4/cluster_heaps
mkdir -p neighborhoods/chr4/cluster_panacus

(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 


: 1

In [6]:
inputOdgi=chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og
outdir=neighborhoods/chr4

while read a b c d; 
do   
    r_path=$(echo "${a}:${b}-${c}")
    r=${r_path}
    nh=${d}
    outprefix=${nh}.region

    ########################## view the pangenome graph of the region ######################
    odgi viz \
    -i $inputOdgi \
    -o $outdir/${outprefix}.bm.png \
    -x 1000 \
    -bm \
    -t 20 \
    -P \
    -r ${r}
    ########################## Node Strandedness ############################################
    odgi viz \
    -i $inputOdgi \
    -o $outdir/${outprefix}.z.png \
    -x 1000 \
    -z \
    -t 20 \
    -P \
    -r ${r}
    
    ############################################ extract the region ############################################
    odgi extract \
    -i $inputOdgi \
    -r ${r} \
    --threads 24 -P \
    -d 100000 \
    -c 1  \
    -o $outdir/cluster_og/${outprefix}.og 
    
    ########################### get the coordinates in all the accessions ########################################
    odgi paths -L \
    -i $outdir/cluster_og/${outprefix}.og  \
    > $outdir/cluster_allPathCoordinates/${outprefix}.allPathsCoordinates.txt
    
    ########################### for bandage plot #################################################################
    odgi view -i $outdir/cluster_og/${outprefix}.og  \
    -g > $outdir/cluster_gfa/${outprefix}.gfa
    
    ########################### Individual path-based PAV comparisons ############################################
    while read line;
    do
        r=${line}
        echo $r|cut -d ':' -f1,2 >tmp1
        echo $r|cut -d ':' -f3|sed "s/-/\t/g" >tmp2
        paste tmp1 tmp2 > \
        $outdir/cluster_allPathCoordinates/${outprefix}.${line}.bed && rm tmp1 && rm tmp2
        
        odgi pav \
        -i $inputOdgi \
        -t 24 \
        -b $outdir/cluster_allPathCoordinates/${outprefix}.${line}.bed > \
        $outdir/cluster_pav/${outprefix}.${line}.pav.txt
        
    done < $outdir/cluster_allPathCoordinates/${outprefix}.allPathsCoordinates.txt
    
    ########################### Node Jumps in Neighborhood ######################################################
    ## for each node how many jumps it has taken
    cat $outdir/cluster_gfa/${outprefix}.gfa \
    |grep "^L" \
    |awk '{print $1,$2,$4,$4-$2}'\
    |sed "s/ /\t/g" \
    > $outdir/cluster_jumps/${outprefix}.nodeJumps.txt
    
    
    ########################### Node Degrees ############################################################
    odgi \
    degree \
    -i $outdir/cluster_og/${outprefix}.og \
    -t 24 \
    -d \
    -r ${r} > $outdir/cluster_jumps/${outprefix}.nodeDegrees.txt
    
    
    ########################### Node Depth ###############################################################
    odgi \
    depth \
    -i $outdir/cluster_og/${outprefix}.og \
    -t 24 \
    -d \
    -r ${r} > $outdir/cluster_jumps/${outprefix}.nodeDepth.txt
    
    ########################### Distance between different Paths (accessions) for the region #############
    odgi \
    similarity \
    -i $outdir/cluster_og/${outprefix}.og \
    -t 24 \
    -d > $outdir/cluster_similarity/${outprefix}.distance.txt
    
    ###### Pangenome Graph Growth  Heaps' Law #########

    odgi sort \
    -i $outdir/cluster_og/${outprefix}.og \
    -p Y \
    -O \
    -t 24 \
    -P \
    -o $outdir/cluster_og/${outprefix}.optimized_sorted.og 
    
    for i in {1..18}
    do
    seq -w 1 ${i} > $outdir/cluster_heaps/pathGroup${i}.groups.txt
    done
    
    
    for i in {1..18}
    do
    odgi paths -i $outdir/cluster_og/${outprefix}.optimized_sorted.og \
    -L|sort -R|head -n ${i} > $outdir/cluster_heaps/pathGroup${i}.txt 
    done
    
    for i in {1..18}
    do
    paste $outdir/cluster_heaps/pathGroup${i}.txt $outdir/cluster_heaps/pathGroup${i}.groups.txt > tmp
    mv tmp $outdir/cluster_heaps/pathGroup${i}.txt
    done
    
    for i in {1..18}
    do
        odgi \
        heaps \
        -i $outdir/cluster_og/${outprefix}.optimized_sorted.og \
        -t 24 \
        -S \
        -d 2 \
        -n 100 \
        -p $outdir/cluster_heaps/pathGroup${i}.txt \
        > $outdir/cluster_heaps/${outprefix}.pathGroup${i}.optimized_sorted.heaps.txt
    done
    
    
    ########################## Panacus ##############################
    parentDir=$outdir
    
    conda activate /u/gshirsekar/conda-envs/pggb #panacus is installed here

    grep '^P' ${parentDir}/cluster_gfa/${outprefix}.gfa|cut -f2|sed "s/_.*//g" > ${parentDir}/cluster_panacus/${outprefix}.groupNames.txt
    grep '^P' ${parentDir}/cluster_gfa/${outprefix}.gfa|cut -f2> ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt
    
    paste ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt \
    ${parentDir}/cluster_panacus/${outprefix}.groupNames.txt \
    > ${parentDir}/cluster_panacus/${outprefix}.groups.txt
    
    sed -i "s/_1_chr4/#1#chr4/g" ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt 
    sed -i "s/_1_chr4/#1#chr4/g" ${parentDir}/cluster_panacus/${outprefix}.groups.txt
    
    
    cat ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt |cut -f1,3 -d ":" > \
    ${parentDir}/cluster_panacus/${outprefix}.newPathNames.txt
    
    paste ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt \
    ${parentDir}/cluster_panacus/${outprefix}.newPathNames.txt > ${parentDir}/cluster_panacus/tmp
    
    while read a b ; do sed -i "s/${a}/${b}/" ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt ; done < ${parentDir}/cluster_panacus/tmp
    while read a b ; do sed -i "s/${a}/${b}/" ${parentDir}/cluster_panacus/${outprefix}.groups.txt ; done < ${parentDir}/cluster_panacus/tmp
    cp ${parentDir}/cluster_gfa/${outprefix}.gfa ${parentDir}/cluster_panacus/${outprefix}.ed.gfa
    sed -i "s/_1_chr4/#1#chr4/g" ${parentDir}/cluster_panacus/${outprefix}.ed.gfa
    
    while read a b ; do sed -i "s/${a}/${b}/g" ${parentDir}/cluster_panacus/${outprefix}.ed.gfa ; done < ${parentDir}/cluster_panacus/tmp 
    
    panacus histgrowth -t24 -q 0,0.1,0.5,1 -l 1,2,3,17 -g ${parentDir}/cluster_panacus/${outprefix}.groups.txt \
    -s ${parentDir}/cluster_panacus/${outprefix}.pathNames.txt \
    ${parentDir}/cluster_panacus/${outprefix}.ed.gfa > ${parentDir}/cluster_panacus/${outprefix}.histgrowth.txt
    
    panacus-visualize -e ${parentDir}/cluster_panacus/${outprefix}.histgrowth.txt > ${parentDir}/cluster_panacus/${outprefix}.histgrowth.node.pdf

done < neighborhoods/all_neighborhoods_at9852.chr4.txt





(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
[odgi::viz] Path range to pangenomic range conversion.
[odgi::viz] Visualizing the graph in the pangenomic range [2.81842e+06, 2.89723e+06]
[odgi::viz] Binned mode
[odgi::viz] bin width: 78.812
[odgi::viz] image width: 1000
[odgi::viz] Path range to pangenomic range conversion.
[odgi::viz] Visualizing the graph in the pangenomic range [2.81842e+06, 2.89723e+06]
[odgi::viz] Binned mode
[odgi::viz] bin width: 78.812
[odgi::viz] image width: 1000
[odgi::extract] expansion and adding connecting edges
[odgi::extract] extracting path ranges 100.00% @ 4.52e+00/s elapsed: 00:00:00:00 remain: 00:00:00:00
[odgi::extract] merge subpaths closer than 100000 bps - iteration 1 (max 3) 100.00% @ 6.99e+00/s elapsed: 00:00:00:02 remain: 00:00:00:00
[odgi::extract] merge subpaths closer than 100000 bps - iteration 2 (max 3) 100.00% @ 6.90e+00/s elapsed: 00:00:00:02 remain: 00:00:00:00
[odgi::extract] adding connecting edges 100.00% @ 2.42e+04/s elapsed: 00:00:

: 1